## Pytorch分布式API

对于进阶炼丹师，仅了解分布式训练是怎么启动的还不够。了解Pytorch分布式训练的一些API，能够帮助我们在自行编写一些需要分布式操作的代码时，更加游刃有余。

PyTorch中的分布式通信函数有很多，其中一些包括：

`torch.distributed.init_process_group()`, 

`torch.distributed.barrier()`, 

`torch.distributed.broadcast()`, 

`torch.distributed.all_reduce()`, 

`torch.distributed.reduce()`,

`torch.distributed.all_gather()`, 

`torch.distributed.gather()`, 

`torch.distributed.scatter()`

等。这些函数可以用于在分布式环境中进行模型训练和数据处理等任务。

### All_reduce 操作

`all_reduce`是分布式通讯中最重要的一个概念之一。

1. 所谓的`reduce`，就是不同节点各有一份数据，把这些数据汇总到一起，并对这些数据做出一定的计算操作，如汇总后取平均。如果只聚集而不额外计算，则为`gather()`操作。在这里，我们规定各个节点上的这份数据有着相同的shape和data type，并规定汇总的方法是相加。简而言之，就是把各个节点上的一份相同形状的数据相加到一起。
2. 所谓的`all_reduce`，就是在reduce的基础上，把最终的结果发回到各个节点上。
3. 具体的`all_reduce`实现，要看具体的backend。流行的GPU backend NCCL，`all_reduce`的实现就是使用了ring思想。DDP利用all_reduce，来进行不同进程上的梯度的平均操作。

接下来，我们写一小段代码对`all_reduce`的效果进行测试：

In [ ]:
import torch
import torch.distributed as dist
import argparse
import utils

def get_args_parser():
    # 与2-1示例完全相同
    parser = argparse.ArgumentParser('Distributed training')
    # parser.add_argument('--local_rank', type=int, default=0)
    parser.add_argument('--world_size', type=int, default=1)
    parser.add_argument('--backend', type=str, default='nccl')
    parser.add_argument("--dist-url", type=str, default="env://")
    parser.add_argument("--sync-bn", action="store_true")
    
    return parser.parse_args()

def main():
    # 和2-1示例一样地初始化分布式环境
    args = get_args_parser()
    utils.init_distributed_mode(args, disable_print=False)
    
    rank = utils.get_rank()
    torch.manual_seed(rank + 123)
    t = torch.randint(0, 10, (2,3)).to("cuda")
    print(f"Rank {rank} has tensor {t}")
    print("---------------------------------------------------")
    
    dist.all_reduce(t, op=dist.ReduceOp.SUM)
    print(f"Rank {rank} has reduced tensor {t}")

这里，我们利用rank来给每张卡初始化一个随机数种子，再让他们产生一个随机的tensor，然后我们利用`dist.all_reduce`函数来测试all_reduce后的结果。我们所预想的状况为，all_reduce后，每张卡上tensor`t`的值都应该是最开始三张卡上`t`的值的累加。接下来我们用torchrun，跑三张卡的实验测试一下：

In [2]:
!torchrun --nproc_per_node=3 test_all_reduce.py

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
| distributed init (rank 0): env://
| distributed init (rank 2): env://
| distributed init (rank 1): env://
Rank 2 has tensor tensor([[5, 9, 0],
        [5, 9, 3]], device='cuda:2')Rank 1 has tensor tensor([[4, 6, 3],
        [5, 6, 1]], device='cuda:1')

---------------------------------------------------
---------------------------------------------------
Rank 0 has tensor tensor([[2, 9, 2],
        [0, 0, 2]], device='cuda:0')
---------------------------------------------------
Rank 1 has reduced tensor tensor([[11, 24,  5],
        [10, 15,  6]], device='cuda:1')
Rank 2 has reduced tensor tensor([[11, 24,  5],
        [10, 15,  6]], device='cuda:2')
Rank 0 has reduced tensor tensor([[11, 24,  

可以发现结果是符合我们的预期的。利用`all_reduce`，可以解决示例2-1内验证精度时数据同步的问题。

比如，针对2-1代码测试accuracy时数据不对的问题，我们在utils.py里面先定义一个执行`all_reduce`的函数：

In [ ]:
def reduce_across_processes(val):
    if not is_dist_avail_and_initialized():
        # nothing to sync, but we still convert to tensor for consistency with the distributed case.
        return torch.tensor(val)

    t = torch.tensor(val, device="cuda")
    dist.barrier()
    dist.all_reduce(t)
    return t

然后，我们对2-1代码检查accuracy的部分作出如下修改：

In [ ]:
'''--------Line 65--------
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        acc_sum = utils.reduce_across_processes(acc_sum)  # 只加了这一行
        print("Epoch: {}, Loss: {:.2f}, acc: {:.2f}, time cost: {:.2f}s".format(epoch, loss_sum/len(train_loader), acc_sum/len(trainset), time.time()-t0))
        
--------Line 73--------'''

现在我们跑一下程序测试一下：

In [3]:
!torchrun --nproc_per_node=8 main.py

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
| distributed init (rank 0): env://
| distributed init (rank 7): env://
| distributed init (rank 2): env://
| distributed init (rank 1): env://
| distributed init (rank 4): env://
| distributed init (rank 3): env://
| distributed init (rank 6): env://
| distributed init (rank 5): env://
Cifar dataset already exist in './data', skip download
/opt/conda/lib/python3.8/site-packages/torch/nn/parallel/scatter_gather.py:9: UserWarning: is_namedtuple is deprecated, please use the python checks instead
  warnings.warn("is_namedtuple is deprecated, please use the python checks instead")
/opt/conda/lib/python3.8/site-packages/torch/nn/parallel/scatter_gather.py:9: UserWarning: is_namedtuple is deprecated, p

现在的准确率便是和单卡时一致的结果，而不是0.0几的错误结果了。

### All_gather 操作

也是用来在分布式计算中收集张量的函数，它的语法如下：

```py
torch.distributed.all_gather(tensor_list, tensor, group=None)
```

其中，tensor_list是一个包含所有进程的张量列表，tensor是当前进程的张量。这个函数会将所有进程的张量收集到tensor_list中，返回值是一个包含所有进程张量的列表。

例如，我们可以模仿`all_reduce`，写一小段对于`all_gather`进行测试的代码：（完整代码见[test_all_gather.py](./test_all_gather.py)）：

In [ ]:
def main():
    args = get_args_parser()
    utils.init_distributed_mode(args, disable_print=False)
    
    rank = utils.get_rank()
    torch.manual_seed(rank + 123)
    t = torch.randint(0, 10, (2,3)).to("cuda")
    print(f"Rank {rank} has tensor {t}")
    print("---------------------------------------------------")
    
    tensor_list = [torch.ones_like(t) for _ in range(dist.get_world_size())]
    dist.all_gather(tensor_list, t)
    print(f"Rank {rank} has gathered tensor {tensor_list}")

然后，我们同样用三张GPU做测试：

In [3]:
!torchrun --nproc_per_node=3 test_all_gather.py

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further tune the variable for optimal performance in your application as needed. 
*****************************************
| distributed init (rank 1): env://
| distributed init (rank 2): env://
| distributed init (rank 0): env://
Rank 0 has tensor tensor([[2, 9, 2],
        [0, 0, 2]], device='cuda:0')
---------------------------------------------------
Rank 2 has tensor tensor([[5, 9, 0],
        [5, 9, 3]], device='cuda:2')Rank 1 has tensor tensor([[4, 6, 3],
        [5, 6, 1]], device='cuda:1')

------------------------------------------------------------------------------------------------------

Rank 1 has gathered tensor [tensor([[2, 9, 2],
        [0, 0, 2]], device='cuda:1'), tensor([[4, 6, 3],
        [5, 6, 1]], device='cuda:1'), tensor([[5, 9, 0],
        [5, 9, 3]], device='cuda:1')]
Rank 2 has gathered t

可见，执行`all_gather()`后，变量`tensor_list`收集了各张卡上所有的tensor`t`，并将其汇总到了这个list里面。

### DistributedSampler

分布式数据采样器是数据并行中的很重要一部分。不同进程所处理的数据集不能重叠，但每个epoch都会随机shuffle数据集来保证训练的性能。`DistributedSampler`利用随机数种子，达到两种shuffle的效果：

1. 横向来看，每个进程对于整个数据集（如ImageNet）的shuffle种子是固定的，即确保每个进程所处理的都是整个数据集互不重叠的子集；

2. 纵向来看，`DistributedSampler`把每个epoch的数值设置成采样种子，以这种方法来在训练迭代的过程中不断shuffle整个ImageNet，先整体按epoch数值shuffle，然后再针对每个进程固定shuffle。


#### RASamlpler

在Facebook发布的Deit论文中，他们对于分布式数据集采样提出了一种更新颖的方法：RASampler。

https://github.com/facebookresearch/deit
https://github.com/facebookresearch/deit/blob/main/samplers.py